In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/learn-co-students/bsc-phase-two-project/main/data/kc_house_data.csv')

In [3]:
df = df[['price', 'sqft_living', 'grade', 
         'yr_built', 'zipcode', 'bathrooms', 'view', 'bedrooms', 'condition', 
         'waterfront', 'floors', 'sqft_basement', 'condition']]
df

,price,sqft_living,grade,yr_built,zipcode,bathrooms,view,bedrooms,condition,waterfront,floors,sqft_basement,condition
0,221900.0,1180,7,1955,98178,1.00,0.0,3,3,NaN,1.0,0.0,3
1,538000.0,2570,7,1951,98125,2.25,0.0,3,3,0.0,2.0,400.0,3
2,180000.0,770,6,1933,98028,1.00,0.0,2,3,0.0,1.0,0.0,3
3,604000.0,1960,7,1965,98136,3.00,0.0,4,5,0.0,1.0,910.0,5
4,510000.0,1680,8,1987,98074,2.00,0.0,3,3,0.0,1.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,360000.0,1530,8,2009,98103,2.50,0.0,3,3,0.0,3.0,0.0,3
21593,400000.0,2310,8,2014,98146,2.50,0.0,4,3,0.0,2.0,0.0,3
21594,402101.0,1020,7,2009,98144,0.75,0.0,2,3,0.0,2.0,0.0,3
21595,400000.0,1600,8,2004,98027,2.50,0.0,3,3,NaN,2.0,0.0,3


In [4]:
df['waterfront'] = df['waterfront'].fillna(0)
df['view'] = df['view'].fillna(0)

In [5]:
df['price'] = df['price'].astype(int)
df['bathrooms'] = df['bathrooms'].astype(int)
# df['view'] = df['view'].astype(int) #has null values
df['floors'] = df['floors'].astype(int)
df['view'] = df['view'].astype(int)
df['waterfront'] = df['waterfront'].astype(int)

In [6]:
#dropped rows with ? in the sqft_basement column
df = df[~df['sqft_basement'].isin(['?'])]

In [7]:
# Isolate independent and depdentent variables 
X = df.drop('price', axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

In [8]:

#OHE for condition (1-5 scale)
# Initialize a One Hot Encoder
ohe = OneHotEncoder(sparse=False)

# Fir the encoder to the training data
ohe.fit(X_train[['condition']])

# Transform the borough column
train_condition = ohe.transform(X_train[['condition']])
             
# Turn the transformed data into a dataframe
# so we can add it to the end of our training dataframe
train_condition = pd.DataFrame(train_condition, 
                        # This is how you get
                        # the feature name from
                        # a one hot encoder              # Set the index so it matched the training data
                        columns=ohe.get_feature_names(), index=X_train.index)

# Add the transformed columns to the X_train dataframe
train_condition = pd.concat([X_train.drop('condition', axis = 1), train_condition], axis = 1)

# Transform the testing data
test_condition = ohe.transform(X_test[['condition']])

# Turn the transformed data into a dataframe
test_condition = pd.DataFrame(test_condition, 
                        columns=ohe.get_feature_names(), index=X_test.index)

# Add the transformed columns to the X_test dataframe
test_condition = pd.concat([X_test.drop('condition', axis = 1), test_condition], axis = 1)


In [9]:
#OHE column names 
ohe.get_feature_names().tolist()

['x0_1',
 'x0_2',
 'x0_3',
 'x0_4',
 'x0_5',
 'x1_1',
 'x1_2',
 'x1_3',
 'x1_4',
 'x1_5']

In [10]:
condition = ['x0_1', 'x0_2', 'x0_3', 'x0_4', 'x0_5']

In [11]:
#OHE for zipcode 
# Initialize a One Hot Encoder
ohe = OneHotEncoder(sparse=False)

# Fir the encoder to the training data
ohe.fit(X_train[['zipcode']])

# Transform the borough column
train_zipcode = ohe.transform(X_train[['zipcode']])
             
# Turn the transformed data into a dataframe
# so we can add it to the end of our training dataframe
train_zipcode = pd.DataFrame(train_zipcode, 
                        # This is how you get
                        # the feature name from
                        # a one hot encoder              # Set the index so it matched the training data
                        columns=ohe.get_feature_names(), index=X_train.index)

# Add the transformed columns to the X_train dataframe
train_zipcode = pd.concat([X_train.drop('zipcode', axis = 1), train_zipcode], axis = 1)


# Transform the testing data
test_zipcode = ohe.transform(X_test[['zipcode']])

# Turn the transformed data into a dataframe
test_zipcode = pd.DataFrame(test_zipcode, 
                        columns=ohe.get_feature_names(), index=X_test.index)
# Add the transformed columns to the X_test dataframe
test_zipcode = pd.concat([X_test.drop('zipcode', axis = 1), test_zipcode], axis = 1)

test_zipcode = pd.concat([test_zipcode, test_condition], axis = 1)
train_zipcode = pd.concat([train_zipcode, train_condition], axis = 1)


In [12]:
zipcode = ['x0_98001','x0_98002','x0_98003','x0_98004','x0_98005','x0_98006','x0_98007','x0_98008','x0_98010','x0_98011','x0_98014','x0_98019','x0_98022','x0_98023','x0_98024','x0_98027','x0_98028','x0_98029','x0_98030','x0_98031','x0_98032','x0_98033','x0_98034','x0_98038','x0_98039','x0_98040','x0_98042','x0_98045','x0_98052','x0_98053','x0_98055','x0_98056','x0_98058','x0_98059','x0_98065','x0_98070','x0_98072','x0_98074','x0_98075','x0_98077','x0_98092','x0_98102','x0_98103','x0_98105','x0_98106','x0_98107','x0_98108','x0_98109','x0_98112','x0_98115','x0_98116','x0_98117','x0_98118','x0_98119','x0_98122','x0_98125','x0_98126','x0_98133','x0_98136','x0_98144','x0_98146','x0_98148','x0_98155','x0_98166','x0_98168','x0_98177','x0_98178','x0_98188','x0_98198','x0_98199']

In [13]:
X = df.drop('price', axis = 1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 12)

In [14]:
model_7_train = X_train[['sqft_living', 'grade',  
                        'bathrooms', 'view', 'yr_built', 'sqft_basement', 'zipcode', 'condition',
                         'bedrooms', 'waterfront','floors']]

model_7_test = X_test[['sqft_living', 'grade',
                          'bathrooms', 'view', 'yr_built', 'sqft_basement', 'zipcode', 'condition',
                         'bedrooms', 'waterfront','floors']]

model_7_train['price'] = y_train
model_7_train = model_7_train[np.abs(stats.zscore(model_7_train['price'])) < 1]
y_train = model_7_train.price
model_7_train.drop('price', axis=1, inplace=True)

model7 = LinearRegression()
model7.fit(model_7_train, y_train)

train_preds = model7.predict(model_7_train)
test_preds = model7.predict(model_7_test)

train_error = mean_squared_error(y_train, train_preds, squared= False)
test_error = mean_squared_error(y_test, test_preds, squared= False)

print('Train error:', train_error)
print('Test error:', test_error)

Train error: 126380.9202618552
Test error: 243932.81486939767


In [15]:
binned = pd.qcut(y_train, q=8)
binned.value_counts()

(167999.999, 260000.0]    1820
(315000.0, 370000.0]      1789
(580000.0, 700000.0]      1787
(430000.0, 499950.0]      1772
(499950.0, 580000.0]      1766
(370000.0, 430000.0]      1760
(700000.0, 915000.0]      1757
(260000.0, 315000.0]      1727
Name: price, dtype: int64

In [16]:
binned = pd.DataFrame(binned)

In [17]:
binned['error'] = y_train - model7.predict(model_7_train)

In [18]:
binned.shape

(14178, 2)

In [19]:
# shows average error for each price range
binned.groupby('price').mean()['error']


price
(167999.999, 260000.0]   -115559.155866
(260000.0, 315000.0]      -90404.127300
(315000.0, 370000.0]      -65451.385452
(370000.0, 430000.0]      -26421.361529
(430000.0, 499950.0]        5018.853331
(499950.0, 580000.0]       47054.981085
(580000.0, 700000.0]       88512.795148
(700000.0, 915000.0]      159291.318589
Name: error, dtype: float64